In [3]:
import cv2
import numpy as np
from numpy import cos, sin, deg2rad, roll, logical_and, where, inner, exp, rad2deg, arctan2, trace, dot, convolve, sqrt, subtract, log, floor, stack, delete, concatenate, max
from numpy.linalg import det, lstsq, norm
from einops import rearrange
from sift import sift
from stero import matchKeyPoints
from random import shuffle
from svd import svd

In [ ]:
# Load the images
image1 = cv2.imread('images/test1.jpeg', cv2.IMREAD_GRAYSCALE)
image2 = cv2.imread('images/test2.jpeg', cv2.IMREAD_GRAYSCALE)

# reduce image size
xscale = 0.25
yscale = 0.25
image1_small = cv2.resize(image1, (0,0), fx = xscale, fy = yscale)
image2_small = cv2.resize(image2, (0,0), fx = xscale, fy = yscale)

# Initialize SIFT detector
sift = cv2.SIFT_create()

# Detect keypoints and compute descriptors
keypoints1, descriptors1 = sift.detectAndCompute(image1, None)
keypoints2, descriptors2 = sift.detectAndCompute(image2, None)

# keypoints1, descriptors1 = sift(image1_small)
# keypoints2, descriptors2 = sift(image2_small)

In [3]:
good_matches = matchKeyPoints(descriptors1, descriptors2)

In [4]:
# Draw matches
matched_image = cv2.drawMatches(image1, keypoints1, image2, keypoints2, good_matches, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
matched_image_small = cv2.resize(matched_image, (0,0), fx = 0.5, fy = 0.5)

# Display the matched image
cv2.imshow('Matches', matched_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [18]:
# rewriting constrian equation from v'*f_m*v = 0 to A*f_v = 0, where f_m is fundamental matrix
# and f_v is the unrolled fundamental matrix
A = np.zeros((8,9))
for i in range(8):
    image1_idx = good_matches[i].queryIdx
    image2_idx = good_matches[i].trainIdx
    u1, v1 = keypoints1[image1_idx].pt
    u2, v2 = keypoints2[image1_idx].pt
    A[i,:] = np.array([u2*u1, u2*v1, u2, v2*u1, v2*v1, v2, u1, v1, 1])

# Solve for non-trival nullspace of matrix A
U, S, V = svd(A)
f_v = V[:,-1]
f_m = f_v.reshape((3,3)) # note: rank of f_m is already 2

# decompose fundamental matrix to get essential matrix
# need camer intrinsic here


(7, 7)


In [26]:
a = np.arange(5)
b = (a**2 + a**2)
count = (b < 9).sum()
count

3

In [19]:
import cv2
import numpy as np

def load_images(image_paths):
    """Load a sequence of images."""
    images = [cv2.imread(path, cv2.IMREAD_GRAYSCALE) for path in image_paths]
    return images

def detect_and_match_features(img1, img2):
    """Detect and match features between two images."""
    orb = cv2.ORB_create()
    kp1, des1 = orb.detectAndCompute(img1, None)
    kp2, des2 = orb.detectAndCompute(img2, None)
    
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = bf.match(des1, des2)
    matches = sorted(matches, key=lambda x: x.distance)
    
    pts1 = np.float32([kp1[m.queryIdx].pt for m in matches])
    pts2 = np.float32([kp2[m.trainIdx].pt for m in matches])
    return pts1, pts2

def compute_rectification_homographies(F, img1, img2, pts1, pts2):
    """Compute the rectification homographies."""
    h, w = img1.shape
    _, H1, H2 = cv2.stereoRectifyUncalibrated(pts1, pts2, F, (w, h))
    return H1, H2

def apply_homographies(img1, img2, H1, H2):
    """Apply the rectifying homographies to the images."""
    h, w = img1.shape
    rectified_img1 = cv2.warpPerspective(img1, H1, (w, h))
    rectified_img2 = cv2.warpPerspective(img2, H2, (w, h))
    return rectified_img1, rectified_img2

def main():
    image_paths = ['images/card1.jpeg', 'images/card2.jpeg']
    img1, img2 = load_images(image_paths)

    pts1, pts2 = detect_and_match_features(img1, img2)
    
    # Estimate fundamental matrix
    F, mask = cv2.findFundamentalMat(pts1, pts2, cv2.FM_RANSAC)
    pts1 = pts1[mask.ravel() == 1]
    pts2 = pts2[mask.ravel() == 1]
    
    # Compute rectification homographies
    H1, H2 = compute_rectification_homographies(F, img1, img2, pts1, pts2)
    
    # Apply homographies to rectify the images
    rectified_img1, rectified_img2 = apply_homographies(img1, img2, H1, H2)

    xscale = 0.5
    yscale = 0.5
    
    rectified_img1_small = cv2.resize(rectified_img1, (0,0), fx = xscale, fy = yscale)
    rectified_img2_small = cv2.resize(rectified_img2, (0,0), fx = xscale, fy = yscale)
    
    # Display the rectified images
    cv2.imshow('Rectified Image 1', rectified_img1_small)
    cv2.imshow('Rectified Image 2', rectified_img2_small)
    cv2.waitKey(0)
    cv2.destroyAllWindows()



In [21]:
main()

In [23]:
import numpy as np
from einops import rearrange
a, b = np.ones((30,3)), np.zeros((30,3))


In [28]:
a = np.array([[1,2,3],[4,5,6],[7,8,9]])
b = np.array([[11,22,33],[44,55,66]])
c = b
np.sum(((b@a)*c), axis=1)

array([ 27588, 150645])

In [34]:
x = rearrange(b[0,:], 'a -> a 1')
y = rearrange(x, 'a b -> b a')
y@a@x


array([[27588]])